In [1]:
# %load Utils/preprocess_util.py
import numpy as np
import pandas as pd
import scipy
import scipy.signal
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim  as optim
from sklearn.model_selection import train_test_split
from Utils.preprocess_util import *

def exponential_running_demean(data, factor=0.001):
    """
    computes exponential moving mean  for each channel given by the formula in https://arxiv.org/abs/1703.05051
    data: 2darray (time, channels)
    """
    df = pd.DataFrame(data)
    meaned = df.ewm(alpha=factor_new).mean()
    demeaned = df - meaned
    demeaned = np.array(demeaned)
    
    return demeaned
def exponential_running_standardize(data, factor_new=0.001,
                                     eps=1e-4):
    '''
    Perform exponential running standardization.

    Compute the exponental running mean :math:`m_t` at time `t` as
    :math:`m_t=\mathrm{factornew} \cdot mean(x_t) + (1 - \mathrm{factornew}) \cdot m_{t-1}`.

    Then, compute exponential running variance :math:`v_t` at time `t` as
    :math:`v_t=\mathrm{factornew} \cdot (m_t - x_t)^2 + (1 - \mathrm{factornew}) \cdot v_{t-1}`.

    Finally, standardize the data point :math:`x_t` at time `t` as:
    :math:`x'_t=(x_t - m_t) / max(\sqrt{v_t}, eps)`.


    Parameters
    ----------
    data: 2darray (time, channels)
    factor_new: float
    eps: float
        Stabilizer for division by zero variance.
    Returns
    -------
    standardized: 2darray (time, channels)
        Standardized data.
    '''
    df = pd.DataFrame(data)
    meaned = df.ewm(alpha=factor_new).mean()
    demeaned = df - meaned
    squared = demeaned * demeaned
    square_ewmed = squared.ewm(alpha=factor_new).mean()
    standardized = demeaned / np.maximum(eps, np.sqrt(np.array(square_ewmed)))
    standardized = np.array(standardized)

    return standardized

def load_preprocess_eeg_data():
    
    X_test = np.load("X_test.npy")[:,0:22,:]
    y_test = np.load("y_test.npy")
    person_train_valid = np.load("person_train_valid.npy")
    X_train_valid = np.load("X_train_valid.npy")[:,0:22,:]
    y_train_valid = np.load("y_train_valid.npy")
    person_test = np.load("person_test.npy")
    
    X_train, X_valid, y_train, y_valid = train_test_split(  X_train_valid, y_train_valid, test_size=0.33,                    random_state=42)
    
    #standardize every data point
    X_train_modified =[]
    X_valid_modified =[]
    X_test_modified =[]

    for xi in X_train:
        X_train_modified.append(exponential_running_standardize(xi.T, eps=1e-4))

    for xi in X_valid:
        X_valid_modified.append(exponential_running_standardize(xi.T, eps=1e-4))

    for xi in X_test:
        X_test_modified.append(exponential_running_standardize(xi.T, eps=1e-4))


    X_train = np.array(X_train_modified)
    X_valid = np.array(X_valid_modified)
    X_test = np.array(X_test_modified)
    
    print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
    print ('Test data shape: {}'.format(X_test.shape))
    print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
    print ('Test target shape: {}'.format(y_test.shape))
    print ('Person train/valid shape: {}'.format(person_train_valid.shape))
    print ('Person test shape: {}'.format(person_test.shape))
    print()
    
    #remmoving eog sginals
    X_train = np.transpose(X_train,[0,2,1])
    X_valid = np.transpose(X_valid,[0,2,1])
    X_test = np.transpose(X_test,[0,2,1])
    print ('Training data shape: {}'.format(X_train.shape))
    print ('Valid data shape: {}'.format(X_valid.shape))
    print ('Training target shape: {}'.format(y_train.shape))
    print ('Valid target shape: {}'.format(y_valid.shape))
    
    #converting labels to range 0-No. of classes
    Y_train = []
    for y in y_train:
        value = np.abs(769-y)
        Y_train.append(value)
    Y_train = np.array(Y_train)

    Y_valid = []
    for y in y_valid:
        value = np.abs(769-y)
        Y_valid.append(value)
    Y_valid = np.array(Y_valid)

    Y_test = []
    for y in y_test:
        value = np.abs(769-y)
        Y_test.append(value)
    Y_test = np.array(Y_test)
    
    return X_train,X_valid,X_test,Y_train,Y_valid,Y_test

def get_accuracy(ouput, target, batch_size):
    ''' Obtain accuracy for training round '''
    
    
    classes_predicted = torch.max(ouput, 1)[1]
    corrects = (np.equal(classes_predicted.tolist(),target.tolist()).astype(int)).sum()

    #corrects = (max_values[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

def threeD_to_fourDTensor(X):
    return Variable(torch.tensor(X.reshape((X.shape[0],1,X.shape[1],X.shape[2],))))

In [2]:
X_train,X_valid,X_test,Y_train,Y_valid,Y_test = load_preprocess_eeg_data()

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 1000, 22)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)

Training data shape: (1417, 22, 1000)
Valid data shape: (698, 22, 1000)
Training target shape: (1417,)
Valid target shape: (698,)


In [3]:
# create feature and targets tensor for train set.  we create tensor, then we will create variable
featuresTrain = torch.from_numpy(X_train)
targetsTrain = torch.from_numpy(Y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
featuresTest = torch.from_numpy(X_test)
targetsTest = torch.from_numpy(Y_test).type(torch.LongTensor) # data type is long

featuresValid = torch.from_numpy(X_valid)
targetsValid = torch.from_numpy(Y_valid).type(torch.LongTensor)

In [4]:
class LSTMModel(nn.Module):
    def __init__(self, batch_size, n_steps, n_inputs, n_neurons, n_outputs):
        super(LSTMModel, self).__init__()

        self.n_neurons = n_neurons
        self.batch_size = batch_size
        self.n_steps = n_steps
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        
        # 2 is the number of lstms stacked vertically(num_layers)
        self.lstm = nn.LSTM(self.n_inputs, self.n_neurons,2) 
       
        self.FC = nn.Linear(self.n_neurons, self.n_outputs)
        
        
    
    def init_hidden(self,):
            # (num_layers, batch_size, n_neurons)
            # 2 is the number of lstms stacked vertically(num_layers)
            return (torch.zeros(2, self.batch_size, self.n_neurons))

    def forward(self, X):
            # transforms X to dimensions: n_steps X batch_size X n_inputs
            X = X.permute(1, 0, 2) 
            self.batch_size = X.size(1)
            self.hidden = self.init_hidden()
            self.cellstate = self.init_hidden()
            lstm_out, (self.hidden, self.cellstate)= self.lstm(X, (self.hidden,self.cellstate))      
            out = self.FC(self.hidden[1])

            return out.view(-1, self.n_outputs) # batch_size X n_output


In [5]:
import torch.utils.data

batch_size = 500
# Pytorch train and test sets
train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
valid = torch.utils.data.TensorDataset(featuresValid,targetsValid)
test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
valid_loader = torch.utils.data.DataLoader(valid, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)


In [6]:
N_STEPS = 1000
N_INPUTS = 22
N_NEURONS = 10
N_OUTPUTS = 10
N_EPHOCS = 10

In [7]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
model = LSTMModel(batch_size, N_STEPS, N_INPUTS, N_NEURONS, N_OUTPUTS)

#batch_size X n_steps X  n_inputs
images_modified = images.view(-1, 1000,22)
logits = model(images_modified.float())
print(logits[0:10])

tensor([[-0.1124, -0.1550,  0.3551, -0.1143,  0.3572,  0.1124,  0.1307,  0.0933,
          0.2209, -0.1382],
        [-0.0643, -0.1507,  0.3637, -0.0527,  0.3962,  0.1134,  0.1344,  0.1006,
          0.2613, -0.1413],
        [-0.1157, -0.1721,  0.3556, -0.0928,  0.3608,  0.1124,  0.1247,  0.0872,
          0.2217, -0.1389],
        [-0.0916, -0.1377,  0.3401, -0.1069,  0.3841,  0.1351,  0.1688,  0.0974,
          0.2333, -0.1774],
        [-0.1093, -0.1622,  0.3520, -0.0999,  0.3678,  0.1034,  0.1230,  0.0873,
          0.2241, -0.1320],
        [-0.1000, -0.1159,  0.3542, -0.1227,  0.3917,  0.1349,  0.1660,  0.1090,
          0.2435, -0.1790],
        [-0.1163, -0.1558,  0.3523, -0.1095,  0.3639,  0.1289,  0.1450,  0.0867,
          0.2209, -0.1578],
        [-0.0976, -0.1629,  0.3428, -0.0946,  0.3531,  0.1424,  0.1655,  0.0907,
          0.2340, -0.1611],
        [-0.1075, -0.1705,  0.3777, -0.0621,  0.3651,  0.1006,  0.1067,  0.0839,
          0.2291, -0.1185],
        [-0.1201, -

In [8]:



# batch_size, epoch and iteration
dtype = torch.FloatTensor
n_iters = 10000
num_epochs = n_iters / (len(X_train) / batch_size)
num_epochs = int(num_epochs)

model = LSTMModel(batch_size, N_STEPS, N_INPUTS, N_NEURONS, N_OUTPUTS)

# Cross Entropy Loss 
loss_fn = nn.CrossEntropyLoss().type(dtype)

#batch GD
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
optimizer = torch.optim.Adam(model.parameters(),lr=0.01,amsgrad=False)

# Create RNN

input_dim = 22    # input dimension
seq_dim = 1000 
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    for i, (signals, labels) in enumerate(train_loader):
        train  = Variable(signals.view(-1, seq_dim, input_dim))
        labels = Variable(labels )
        
        # Clear gradients
        optimizer.zero_grad()
        
        # reset hidden states
        model.hidden = model.init_hidden() 
                
        # Forward propagation
        outputs = model(train.float())
        
        # Calculate softmax and cross entropy loss
        loss = loss_fn(outputs, labels)
        
        # Calculating gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
                    
        #print("parameters===",list(model.parameters())[0].data)
                
        if count % 1 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for signals, labels in valid_loader:
                signals = Variable(signals.view(-1, seq_dim, input_dim))
                #print(signals.shape)
                # Forward propagation
                outputs_valid = model(signals.float())

                # Get predictions from the maximum value
                predicted = torch.max(outputs_valid.data, 1)[1]
                
                # Total number of labels
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / float(total)
            
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            if count % 1 == 0:
                # Print Loss
                print('Iteration: {}  Loss: {}  Valid Accuracy: {} %'.format(count, loss.data, accuracy))

Iteration: 1  Loss: 2.3557286262512207  Valid Accuracy: 0 %
Iteration: 2  Loss: 2.3137168884277344  Valid Accuracy: 11 %
Iteration: 3  Loss: 2.2757842540740967  Valid Accuracy: 22 %
Iteration: 4  Loss: 2.2168195247650146  Valid Accuracy: 22 %


KeyboardInterrupt: 

In [19]:

X_valid_tensor = torch.from_numpy(X_valid.reshape(-1, seq_dim, input_dim))
print(X_valid_tensor.shape)
y_pred_valid = model( X_valid_tensor.float())
val_acc = get_accuracy(y_pred_valid, Y_valid,
    batch_size=X_valid.shape[0])
print(val_acc)

torch.Size([698, 1000, 22])
23.35243553008596


In [21]:

X_test_tensor = torch.from_numpy(X_test.reshape(-1, seq_dim, input_dim))
y_pred_test = model( X_test_tensor.float())
test_acc = get_accuracy(y_pred_test, Y_test,
    batch_size=X_test.shape[0])
print(test_acc)

29.345372460496613
